#**Geographic analysis**

1. Analyze the geographical distribution of deals by cities.
2. Examine the impact of the level of German language proficiency on deal success in different cities


In [ ]:
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
pd.set_option('display.max_columns', None) #display all columns (so that they are not hidden with «…»)

In [ ]:
deals = pd.read_pickle('deals_df.pkl')

In [ ]:
deals.head()

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Campaign,SLA,Content,Term,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch,_open_deal,Payment Category,Offer Category,_SLA_hours
0,5805028000056864695,Ben Hall,NaT,Unknown,New Lead,Unknown,03.07.23women,0 days 00:00:00,v16,women,Facebook Ads,Unknown,Unknown,Unknown,2024-06-21 15:30:00,0,0,NaN,NaN,5805028000056849495,Unknown,Unknown,True,Unknown,Unknown,0.000000
1,5805028000056859489,Ulysses Adams,NaT,Unknown,New Lead,Unknown,Unknown,0 days 00:00:00,Unknown,Unknown,Organic,Unknown,Web Developer,Morning,2024-06-21 15:23:00,6,0,0.0,2000.0,5805028000056834471,Unknown,Unknown,True,No Payment,Regular Offer,0.000000
2,5805028000056832357,Ulysses Adams,2024-06-21,D - Non Target,Lost,Non target,engwien_AT,0 days 00:26:43,b1-at,21_06_2024,Telegram posts,Unknown,Unknown,Unknown,2024-06-21 14:45:00,0,0,NaN,NaN,5805028000056854421,Unknown,Unknown,False,Unknown,Unknown,0.445278
3,5805028000056824246,Eva Kent,2024-06-21,E - Non Qualified,Lost,Invalid number,04.07.23recentlymoved_DE,0 days 01:00:03.999999999,bloggersvideo14com,recentlymoved,Facebook Ads,Unknown,Unknown,Unknown,2024-06-21 13:32:00,0,0,NaN,NaN,5805028000056889351,Unknown,Unknown,False,Unknown,Unknown,1.001111
4,5805028000056873292,Ben Hall,2024-06-21,D - Non Target,Lost,Non target,discovery_DE,0 days 00:53:12.000000001,website,Unknown,Google Ads,Unknown,Unknown,Unknown,2024-06-21 13:21:00,0,0,NaN,NaN,5805028000056876176,Unknown,Unknown,False,Unknown,Unknown,0.886667


In [ ]:
# Columns we use
need_cols = ['Id','City','Stage','Level of Deutsch']

###**1. Geography: distribution of deals and conversion by cities**


In [ ]:
# 1.1 Aggregation by cities
df_city = deals[deals['City'].astype(str) != 'Unknown'].copy()

city_agg = (df_city.groupby('City', observed=True).agg(total_deals=('Id','nunique'),
                 success=('Stage', lambda s: (s == 'Payment Done').sum())).reset_index())

city_agg['conversion'] = np.where(city_agg['total_deals']>0, city_agg['success']/city_agg['total_deals']*100, np.nan)

city_agg = city_agg.sort_values('total_deals', ascending=False)

top10_cities = city_agg.head(10).copy()

top10_cities['conv_txt'] = top10_cities['conversion'].round(1).astype(str) + '%'
top10_cities['total_txt'] = top10_cities['total_deals'].astype(int).astype(str)

top10_cities[['City','total_deals','success','conversion']] \
    .round({'conversion': 1}) \
    .style.format({'conversion': '{:.1f}'}) \
    .hide(axis='index')



City,total_deals,success,conversion
Berlin,182,78,42.9
München,74,27,36.5
Hamburg,62,22,35.5
Nürnberg,45,9,20.0
Leipzig,45,18,40.0
Düsseldorf,33,7,21.2
Dresden,28,9,32.1
Frankfurt,27,7,25.9
Dortmund,26,4,15.4
Köln,25,9,36.0


In [ ]:
palette = {
    "yellow": "#FFC300",  # бары (volume)
    "green":    "#2CA02C",  # линия конверсии
    "line":     "#D9D9D9",  # цвет осей
    "char":     "#2E2E2E",  # текст
    "bg":       "rgba(0,0,0,0)"
}

fig = make_subplots(specs=[[{"secondary_y": True}]])

# 1) total deals
fig.add_trace(
    go.Bar(
        x=top10_cities['City'],
        y=top10_cities['total_deals'],
        name='Total Deals',
        marker=dict(color=palette["yellow"], line=dict(color=palette["line"], width=1.0)),
        opacity=0.4,
        text=top10_cities['total_txt'],
        textposition='outside',
        textfont=dict(size=11, color=palette["char"]),
        cliponaxis=False),
    secondary_y=False)

# 2) conversion
fig.add_trace(
    go.Scatter(
        x=top10_cities['City'],
        y=top10_cities['conversion'],
        name='Conversion Rate (%)',
        mode='lines+markers+text',
        line=dict(color=palette["green"], width=2),
        marker=dict(size=8, line=dict(color=palette["line"], width=1)),
        text=top10_cities['conv_txt'],
        textposition='top center',
        textfont=dict(size=11, color=palette["char"]),),
    secondary_y=True)

y1_max = float(top10_cities['total_deals'].max()) * 1.15
y2_max = float(top10_cities['conversion'].max()) * 1.20

fig.update_layout(
    title=dict(text='Top-10 Cities: Volume & Conversion', x=0.0,
               font=dict(size=18, color=palette["char"])),
    barmode='overlay',
    xaxis=dict(
        title='City',
        showgrid=False, zeroline=False,
        tickangle=45,
        linecolor=palette["line"], tickcolor=palette["line"]),
    yaxis=dict(
        title='Deals',
        range=[0, y1_max],
        showgrid=False, zeroline=False,
        linecolor=palette["line"], tickcolor=palette["line"]),
    yaxis2=dict(
        title='Conversion (%)',
        range=[0, y2_max],
        overlaying='y', side='right',
        showgrid=False, zeroline=False),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
    font=dict(family="Arial", color=palette["char"]),
    paper_bgcolor=palette["bg"],
    plot_bgcolor=palette["bg"],
    margin=dict(l=70, r=70, t=70, b=90))

fig.show()


**1. Распределение сделок по городам**

- Топ-10 городов формируют основную часть сделок.

- Берлин лидирует по объему: 182 сделки (43% конверсии).

- Мюнхен, Гамбург, Лейпциг также показывают значительные объемы и конверсии выше среднего уровня.

- Есть города с  низкой конверсией ( Дюссельдорф — 21%, Дортмунд).


###**2. German language proficiency and deal success by city**

**Логика**:

- упорядочим уровни языка (Unknown, A1, A2, B1, B2, C1, C2);

- возьмём Top-10 городов по объёму (чтобы не перегружать графики);

- посчитаем по каждой паре (City × Level): всего сделок, успешные, конверсию;

- покажем две картинки:

-- heatmap конверсии — где какой уровень «работает» лучше,

-- stacked bar — распределение уровней языка внутри города (структура спроса).

In [ ]:
# 2.1 Preparation of language levels and top cities
level_order = ['Unknown','A1','A2','B1','B2','C1','C2']

df_lang = df_city.copy()
df_lang['Level of Deutsch'] = (df_lang['Level of Deutsch'].astype(str).replace({'nan':'Unknown'}))

df_lang['Level of Deutsch'] = pd.Categorical(df_lang['Level of Deutsch'], categories=level_order, ordered=True)

top_cities_names = top10_cities['City'].tolist()
df_lang_top = df_lang[df_lang['City'].isin(top_cities_names)].copy()

In [ ]:
# 2.2 Aggregation (City x Level): volume, successes, conversion
lang_pivot = (df_lang_top.groupby(['City','Level of Deutsch'], observed=True).agg(total=('Id','nunique'), success=('Stage', lambda s: (s=='Payment Done').sum())).reset_index())
lang_pivot['conv'] = np.where(lang_pivot['total']>0, (lang_pivot['success']/lang_pivot['total']*100).round(1), np.nan)

In [ ]:
# 2.3 HEATMAP: Conversion (%) by City × Level
heat = (lang_pivot.pivot(index='City', columns='Level of Deutsch', values='conv').reindex(index=top_cities_names, columns=level_order))

gold_orange = ["#fff9e6", "#ffe9a8", "#ffd36b", "#ffb347", "#ff9933", "#e67300"]

fig_hm = px.imshow(heat, text_auto='.1f', color_continuous_scale=gold_orange, aspect='auto',
    title='Conversion by City × Level of Deutsch (Top-10 cities)',
    labels=dict(color='Conversion (%)'))

fig_hm.update_layout(
    template='plotly_white',
    margin=dict(t=70, r=20, b=50, l=80),
    font=dict(family="Inter, Segoe UI, Roboto, Arial", size=12),
    coloraxis_colorbar=dict(
        title="Conversion (%)",
        ticksuffix="%",
        thickness=12,
        len=0.8))

fig_hm.update_traces(xgap=2, ygap=2, hovertemplate="<b>%{y}</b> × <b>%{x}</b><br>Conversion: %{z:.1f}%<extra></extra>")

fig_hm.update_xaxes(title_text="Level of Deutsch")
fig_hm.update_yaxes(title_text="City")

fig_hm.show()


In [ ]:
# 2.4 STACKED BAR: Language levels mix by city
df_lang_no_unk = df_lang_top[df_lang_top['Level of Deutsch'] != 'Unknown'].copy()

stack = (df_lang_no_unk.groupby(['City','Level of Deutsch'], observed=True)
    .agg(total=('Id','nunique')).reset_index())

level_order_no_unk = ['A1','A2','B1','B2','C1','C2']
stack['Level of Deutsch'] = pd.Categorical(
    stack['Level of Deutsch'], categories=level_order_no_unk, ordered=True)

city_totals = stack.groupby('City', observed=True)['total'].transform('sum')
stack['share'] = (stack['total'] / city_totals * 100).round(1)

gold_gray_map = {
    'A1': '#fff9e6',  # very light gold
    'A2': '#ffe9a8',  # soft gold
    'B1': '#ffd36b',  # warm gold
    'B2': '#cfcfcf',  # light gray
    'C1': '#999999',  # medium gray
    'C2': '#4d4d4d',  # dark gray
}

fig_stack = px.bar(
    stack.sort_values(['City','Level of Deutsch']),
    x='City', y='total', color='Level of Deutsch',
    category_orders={'Level of Deutsch': level_order_no_unk, 'City': top_cities_names},
    color_discrete_map=gold_gray_map,
    title='Language Mix by City (Unknown removed)',
    labels={'total':'Deals', 'City':'City'},
    text='total')

fig_stack.update_layout(
    barmode='stack',
    template='plotly_white',
    margin=dict(t=70, r=20, b=60, l=60),
    font=dict(family="Inter, Segoe UI, Roboto, Arial", size=12),
    legend_title_text='',)

fig_stack.update_traces(
    textposition='inside',
    textfont=dict(size=11),
    marker_line_color='#d9d9d9',
    marker_line_width=0.6,
    hovertemplate=(
        "<b>%{x}</b><br>"
        "Level: %{legendgroup}<br>"
        "Deals: %{y:,}<br>"
        "Share: %{customdata:.1f}%<extra></extra>"),
    customdata=stack['share'].values)

fig_stack.update_xaxes(title_text="City")
fig_stack.update_yaxes(title_text="Deals")

fig_stack.show()


**4. Рекомендации**

 Усилить присутствие в городах с высокой конверсией (Берлин, Мюнхен, Гамбург) и масштабировать локальные кампании.

Локализовать предложения в городах с низкой конверсией, учитывая специфику клиентской базы.

 Фокус на обучении клиентов со знанием A2–B2: таргетированные кампании на эту аудиторию дадут максимальный результат.

 Внедрить интерактивную карту сделок для регулярного мониторинга географических трендов.

 Дополнительно проанализировать причины низкой конверсии у клиентов с высоким уровнем языка (C1–C2), чтобы понять, связаны ли они с продуктом, ценой или другими факторами.